In [3]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from datasets import load_dataset, Dataset

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kuro_ds = load_dataset('Kurosawama/beam_search_DPO', split = 'train')

def initialize_model(model_id):
    quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)
    gen_config = GenerationConfig.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.chat_template = """
            <|im_start|>system
            {SYSTEM}<|im_end|>
            <|im_start|>user
            {INPUT}<|im_ed|>
            <|im_start|>assistant
            {OUTPUT}<|im_end|>
        """
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quant_config, generation_config = gen_config)
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    return model, tokenizer

#model_id = 'meta-llama/Llama-3.2-1B'
#model_id = 'meta-llama/Llama-3.2-1B-Instruct'

base_model_id = 'meta-llama/Llama-2-7b-hf'
align_model_id = 'Kurosawama/Llama-2-7b-DPO-beamsearch-align'

base_model, base_tokenizer = initialize_model(base_model_id)
align_model, align_tokenizer = initialize_model(align_model_id)

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

OSError: Can't load tokenizer for 'Kurosawama/Llama-2-7b-DPO-beamsearch-align'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Kurosawama/Llama-2-7b-DPO-beamsearch-align' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [6]:
kuro_ds

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 340
})

In [29]:
def get_answer(dataset, ans_num):
    """
    dataset = huggingface dataset 
    ans_num = int ; Valor de la entrada del dataset que queremos analizar

    Nos regresa las respuestas del modelo filtradas. 
    """
    chosen = dataset['chosen'][ans_num][1]['content']
    rejected = dataset['rejected'][ans_num][1]['content']
    return chosen, rejected

cho1, rej1 = get_answer(kuro_ds, 0)
print(cho1, '\n', '---', '\n', rej1)

∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))  
 --- 
  Dependent(x) ::: x is a person dependent on caffeine.
             Drinks(x) ::: x regularly drinks coffee.
             Jokes(x) ::: x jokes about being addicted to caffeine.
             Unaware(x) ::: x is unaware that caffeine is a drug.
             Student(x) ::: x is a student.
             Premises:
             ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
             ∀x (Drinks(x) ⊕ Jokes(x)) ::: People who either regularly drink coffee or joke about being addicted to caffeine.
             ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
             (

In [30]:
cho1

'∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) '

In [31]:
# '∀x|∃x'
cho1.split('∀x ')

['',
 '(DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 '(DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 '(¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

In [34]:
import re
re.split('∀x|∃x|¬\(', cho1) # Casi pero no nos regesa la ER.

['',
 ' (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 ' (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 ' (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ',
 'Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ',
 'IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

**VER QUÉ VERGAS VAMOS A HACER PARA LA EVALUACIÓN Y LA SEPARACIÓN DE PREMISAS**